In [1]:
__import__('pysqlite3')
import sys
sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')

# from ibm_watsonx_ai.foundation_models import ModelInference
# from ibm_watsonx_ai.metanames import GenTextParamsMetaNames as GenParams
# from ibm_watsonx_ai.metanames import EmbedTextParamsMetaNames
# from ibm_watsonx_ai import Credentials
# from langchain_ibm import WatsonxLLM, WatsonxEmbeddings

from langchain_openai import AzureOpenAIEmbeddings, AzureOpenAI, AzureChatOpenAI

from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, pipeline
from langchain_huggingface import HuggingFaceEndpoint, ChatHuggingFace, HuggingFacePipeline

from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.callbacks import get_openai_callback
from langchain.chains import RetrievalQA

from fastapi import FastAPI, Response, File, UploadFile, Form
from pydantic import BaseModel

import os
import shutil
import tiktoken
import uvicorn
import gradio as gr

# You can use this section to suppress warnings generated by your code:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn
warnings.filterwarnings('ignore')

/home/codespace/.local/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
## LLM
# os.environ["HUGGINGFACEHUB_API_TOKEN"]
# def get_hg_llm():
#     llm = HuggingFaceEndpoint(
#         repo_id="HuggingFaceH4/zephyr-7b-beta",
#         task="text2text-generation",
#         max_new_tokens=512,
#         do_sample=False,
#         repetition_penalty=1.03,
#     )

#     return llm

# LLM
def get_hg_llm(task="text2text-generation"):
    model_name = "google/flan-t5-large"
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

    text2text_pipeline = pipeline(task, model=model, tokenizer=tokenizer, max_length=2048, device=0)
    llm = HuggingFacePipeline(pipeline=text2text_pipeline)

    return llm

# LLM
def get_az_llm():
    llm = AzureOpenAI(
        api_key=os.getenv('AZURE_OPENAI_API_KEY'),
        api_version=os.getenv('OPENAI_API_VERSION'),
        azure_deployment="gpt-35-turbo-instruct",
        azure_endpoint=os.getenv('AZURE_OPENAI_ENDPOINT'),
        max_tokens=128
    )

    return llm

# Document loader
def document_loader(filename):
    loader = PyPDFLoader(filename)
    loaded_document = loader.load()
    return loaded_document

# Text splitter
def text_splitter(data):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=50,
        length_function=len,
    )
    chunks = text_splitter.split_documents(data)
    return chunks

In [3]:
# Vector db
def vector_database(chunks):
    global DATABASE_TOKENS, VECTOR_DB

    embedding_model = AzureOpenAIEmbeddings(
        model="text-embedding-3-large",
        # dimensions: Optional[int] = None, # Can specify dimensions with new text-embedding-3 models
        azure_endpoint=os.getenv("AZURE_OPENAI_EMB_ENDPOINT"), # If not provided, will read env variable AZURE_OPENAI_ENDPOINT
        api_key=os.getenv("AZURE_OPENAI_EMB_API_KEY"), # Can provide an API key directly. If missing read env variable AZURE_OPENAI_API_KEY
        openai_api_version=os.getenv("AZURE_OPENAI_EMB_API_VERSION"), # If not provided, will read env variable AZURE_OPENAI_API_VERSION
    )

    DATABASE_TOKENS = [len(TOKENIZER.encode(chunk.page_content)) for chunk in chunks]
    if VECTOR_DB != None:
        VECTOR_DB._collection.delete(VECTOR_DB._collection.get()['ids'])

    VECTOR_DB = Chroma.from_documents(chunks, embedding_model)
    return VECTOR_DB

# Retriever
def retriever(filename):
    splits = document_loader(filename)
    chunks = text_splitter(splits)
    vectordb = vector_database(chunks)
    retriever = vectordb.as_retriever()
    return retriever

# QA Chain
def retriever_qa(filename, query):
    llm = get_hg_llm()
    retriever_obj = retriever(filename)

    qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=retriever_obj, return_source_documents=False,)
    response = qa.invoke(query)

    return response['result']

In [ ]:
DATABASE_TOKENS = 0
VECTOR_DB = None

# Create Gradio interface
rag_application = gr.Interface(
    fn=retriever_qa,
    allow_flagging="never",
    inputs=[
        gr.File(label="Upload PDF File", file_count="single", file_types=['.pdf'], type="filepath"),  # Drag and drop file upload
        gr.Textbox(label="Input Query", lines=2, placeholder="Type your question here...")
    ],
    outputs=gr.Textbox(label="Output"),
    title="RAG Chatbot",
    description="Upload a PDF document and ask any question. The chatbot will try to answer using the provided document."
)

# Launch the app
rag_application.launch(server_name="0.0.0.0", server_port= 7890)

In [ ]:
rag_application.close()